In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
def create_data():
    record = [['青年', '否', '否', '一般', '否'],
              ['青年', '否', '否', '好', '否'],
              ['青年', '是', '否', '好', '是'],
              ['青年', '是', '是', '一般', '是'],
              ['青年', '否', '否', '一般', '否'],
              ['中年', '否', '否', '一般', '否'],
              ['中年', '否', '否', '好', '否'],
              ['中年', '是', '是', '好', '是'],
              ['中年', '否', '是', '非常好', '是'],
              ['中年', '否', '是', '非常好', '是'],
              ['老年', '否', '是', '非常好', '是'],
              ['老年', '否', '是', '好', '是'],
              ['老年', '是', '否', '好', '是'],
              ['老年', '是', '否', '非常好', '是'],
              ['老年', '否', '否', '一般', '否'],
              ]
    tags = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回训练数据集和各特征的名称
    return np.array(record), tags

In [3]:
data, labels = create_data()
train_data = pd.DataFrame(data, columns=labels)

In [4]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [5]:
def calc_ent(data):
    """计算熵"""
    counter = Counter(data[:, -1])  # 不同类的个数
    pro_vector = np.array(list(counter.values())) / len(data)  # 不同类所占的比例
    res = - pro_vector @ np.log2(pro_vector)

    return res

In [6]:
calc_ent(data)  # 经验熵H(D)

0.9709505944546686

In [7]:
def calc_cond_ent(data, col=0):
    """计算条件熵"""
    conter = Counter(data[:, col])  # 特征col不同取值的个数
    data_length = len(data)
    pro_vector = np.array(list(conter.values())) / data_length  # 特征col不同取值所占的比例
    hd_vector = list()
    for i in conter.keys():
        hd_vector.append(calc_ent(data[np.argwhere(data == i)[:, 0]]))
    result = pro_vector @ hd_vector

    return result

In [8]:
calc_cond_ent(data)  # 条件熵H(D|A)

0.8879430945988998

In [9]:
def info_gain(ent, cond_ent):
    """信息增益"""
    return ent - cond_ent


def info_gain_train(data):
    """找出信息增益最大的特征"""
    count = len(data[0]) - 1
    ent = calc_ent(data)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain(ent, calc_cond_ent(data, col=c))  # 不同特征划分的信息增益g(D,A)
        best_feature.append((c, c_info_gain))
        print('特征({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))

    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益最大，选择为根节点特征'.format(labels[best_[0]])

In [10]:
info_gain_train(np.array(data))

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.401
特征(有自己的房子) - info_gain - 0.458
特征(信贷情况) - info_gain - 0.363


'特征(有自己的房子)的信息增益最大，选择为根节点特征'